In [1]:
import os
!pwd

/home/lok/Documents/ML_Projects/rice_leaf_disease/research


In [2]:
os.chdir('../')
!pwd

/home/lok/Documents/ML_Projects/rice_leaf_disease


In [3]:
import zipfile
import urllib.request as request
from dataclasses import dataclass
from pathlib import Path
from kaggle.api.kaggle_api_extended import KaggleApi
from rice_leaf_disease.constants import *
from rice_leaf_disease.utils.common import read_yaml, create_directories, get_size
from rice_leaf_disease import logger

In [4]:
@dataclass(frozen=True)
class DataIngestionConfig:
    """
    Configuration class for the data ingestion phase of the pipeline
    to store configuration details.
    - @dataclass decorator is used to automatically generate special 
    methods for the class such as __init__()
    - The frozen = True parameter makes the class immutable. 
    """
    root_dir: Path
    source_url: str
    local_data_file:Path
    unzip_dir: Path
    

In [5]:
class ConfigurationManager:
    """
    This class handles reading configurations, creating necessary directories,
    and providing step-specific configuration objects.
    """
    
    def __init__(self,config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Retrieves and prepares the configuration specific to 
        the data ingestion step.
        """
        
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [ ]:
class DataIngestion:
    
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    # Have already downloaded the data
    # data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-22 22:20:02,343: INFO : common : YAML file: config/config.yaml loaded successfully!]
[2024-12-22 22:20:02,349: INFO : common : YAML file: params.yaml loaded successfully!]
[2024-12-22 22:20:02,353: INFO : common : created directory at: artifacts]
[2024-12-22 22:20:02,359: INFO : common : created directory at: artifacts/data_ingestion]
